In [ ]:
import numpy as np
import pandas as pd

def backtest_funding_premium_stopentry(
    df_in: pd.DataFrame,
    price_col="prep_close",
    high_col="perp_high",
    low_col ="perp_low",
    fund_col="funding_rate",
    # --- funding & extremes ---
    base_per_8h = 1e-4,        # 0.01% per 8h (Binance USDT perps)
    pct_win = 90,              # rolling window (bars) for percentiles (~15 days on 4H)
    p_hi = 0.97, p_lo = 0.03,  # extremes
    # --- context windows (past-only) ---
    roll_high_n = 24,          # price 24-bar high for tops
    fund_low_n  = 18,          # funding 18-bar low for bottoms (~3 days on 4H)
    # --- stop-entry trigger ---
    stopentry_buffer = 0.0005, # 0.05% buffer around prior level
    # --- exits ---
    pivot_k = 6,               # swing pivot lookback for structure stop
    struct_buffer = 0.005,     # 0.5% beyond pivot
    struct_confirm_closes = 2, # 2 consecutive closes beyond stop to exit
    struct_grace_bars = 2,     # ignore structure stop for first N bars
    fresh_low_lookahead = 3,   # for long invalidation: funding fresh low within N bars
    time_stop_settlements = 2, # exit after N settlements if nothing else
    cat_stop_pct = 0.065,      # 6.5% catastrophe stop (~ well inside 10x buffer)
    cooldown_bars = 2,         # sit out after exit
    dedup_bars = 12,           # don't re-enter same side within N bars of last entry
    # --- execution & costs ---
    notional_usd = 1_000.0,    # fixed notional per trade
    taker_fee = 0.0004,        # 0.04% per side
    settlement_hours=(0,8,16), # Binance-style 8h schedule
):
    """
    Returns: log (DataFrame), stats (dict)
    Assumes df_in has a UTC datetime index at a regular bar size (e.g., 4H).
    """
    df = df_in.copy()
    # --- sanity on index ---
    if not pd.api.types.is_datetime64_any_dtype(df.index):
        raise ValueError("Index must be datetime (UTC). Do df.set_index('bar_time') first.")
    df = df.sort_index()

    # Aliases
    PX, HI, LO, FR = price_col, high_col, low_col, fund_col

    # --- settlement mask ---
    df["is_settle"] = df.index.hour.isin(settlement_hours) & (df.index.minute == 0)

    # --- funding premium & rolling extremes (adaptive) ---
    df["fund_premium"] = df[FR] - base_per_8h
    roll = df["fund_premium"].rolling(pct_win, min_periods=pct_win)
    df["p_hi"] = roll.quantile(p_hi)
    df["p_lo"] = roll.quantile(p_lo)

    # --- past-only context (NO look-ahead) ---
    df["roll_high_prev_24"] = df[PX].shift(1).rolling(roll_high_n, min_periods=roll_high_n).max()
    df["fund_low_prev_18"]  = df["fund_premium"].shift(1).rolling(fund_low_n, min_periods=fund_low_n).min()

    # --- setups (arm the idea) ---
    setup_short = (df["fund_premium"] >= df["p_hi"]) & (df[PX] >= df["roll_high_prev_24"])
    setup_long  = (df["fund_premium"] <= df["p_lo"])  | (df["fund_premium"] <= df["fund_low_prev_18"])

    # --- stop-entry levels from prior bar extremes ---
    prior_low  = df[LO].shift(1)
    prior_high = df[HI].shift(1)
    stop_short_lvl = prior_low  * (1 - stopentry_buffer)  # sell-stop to short
    stop_long_lvl  = prior_high * (1 + stopentry_buffer)  # buy-stop to long

    # arm now, fill on next bar if crossed
    arm_short = setup_short
    arm_long  = setup_long

    short_fill = arm_short.shift(1) & (df[LO]  <= stop_short_lvl.shift(1))
    long_fill  = arm_long.shift(1)  & (df[HI] >= stop_long_lvl.shift(1))

    # entry price = the armed stop level (from previous bar)
    entry_px_short = stop_short_lvl.shift(1).where(short_fill)
    entry_px_long  = stop_long_lvl.shift(1).where(long_fill)

    entry_side = pd.Series(np.where(short_fill, -1, np.where(long_fill, +1, 0)), index=df.index)
    entry_price = entry_px_short.fillna(entry_px_long)

    # --- helper state for exits ---
    def get_pivot_high(i):
        j0 = max(0, i - pivot_k)
        return float(df[HI].iloc[j0:i+1].max())

    def get_pivot_low(i):
        j0 = max(0, i - pivot_k)
        return float(df[LO].iloc[j0:i+1].min())

    # precompute for funding invalidation thresholds
    # "pinned near extreme at settlement" for shorts/longs
    pinned_hi = df["is_settle"] & (df["fund_premium"] >= df["p_hi"] - 1e-12)
    fresh_low = df["fund_premium"] < df["fund_low_prev_18"] - 1e-12  # fresh multi-day low vs past

    # --- backtest loop ---
    trades = []
    pos = 0              # +1 long, -1 short, 0 flat
    qty = 0.0            # contracts (not used for USD pnl calc; we use notional directly)
    entry_px = np.nan
    entry_i = None
    entry_time = None
    pending_entry_fee = 0.0
    fund_acc_usd = 0.0
    closes_beyond = 0
    struct_stop_lvl = np.nan
    entry_bar_high = np.nan
    entry_bar_low  = np.nan
    next_eligible_i = 0
    last_short_entry_i = -10_000
    last_long_entry_i  = -10_000

    idx = df.index
    N = len(df)

    for i in range(1, N):  # start from 1 because we use shift(1)
        # accumulate funding while in position (only settlements)
        if pos != 0 and df["is_settle"].iloc[i-1]:
            fund_acc_usd += float(df[FR].iloc[i-1]) * notional_usd * pos

        # if flat, check entries (respect cooldown & de-dup)
        if pos == 0 and i >= next_eligible_i:
            side = int(entry_side.iloc[i])
            px_e = entry_price.iloc[i]
            if side != 0 and np.isfinite(px_e):
                # de-dup: ignore same-side entries if too soon since last
                if side == -1 and (i - last_short_entry_i) < dedup_bars:
                    pass
                elif side == +1 and (i - last_long_entry_i) < dedup_bars:
                    pass
                else:
                    # enter
                    pos = side
                    entry_px = float(px_e)
                    entry_i = i
                    entry_time = idx[i]
                    entry_bar_high = float(df[HI].iloc[i])
                    entry_bar_low  = float(df[LO].iloc[i])
                    fund_acc_usd = 0.0
                    pending_entry_fee = taker_fee * notional_usd
                    closes_beyond = 0  # reset for structure stop confirm

                    # structure stop level at entry
                    if pos == -1:
                        pivot = get_pivot_high(i-1)  # pivot BEFORE entry bar
                        struct_stop_lvl = max(pivot * (1 + struct_buffer), entry_px * (1 + struct_buffer/2))
                        last_short_entry_i = i
                    else:
                        pivot = get_pivot_low(i-1)
                        struct_stop_lvl = min(pivot * (1 - struct_buffer), entry_px * (1 - struct_buffer/2))
                        last_long_entry_i = i

                    continue  # proceed to next bar

        # manage open position
        if pos != 0:
            price_close = float(df[PX].iloc[i])
            price_high  = float(df[HI].iloc[i])
            price_low   = float(df[LO].iloc[i])

            # --- catastrophe stop (intrabar) ---
            exit_reason = None
            if pos == -1 and price_high >= entry_px * (1 + cat_stop_pct):
                exit_reason = "cat_short"
            if pos == +1 and price_low  <= entry_px * (1 - cat_stop_pct):
                exit_reason = "cat_long"

            # --- structure stop with grace & 2-close confirmation ---
            if exit_reason is None:
                if (i - entry_i) >= struct_grace_bars:
                    if pos == -1:
                        if price_close > struct_stop_lvl:
                            closes_beyond += 1
                        else:
                            closes_beyond = 0
                        if closes_beyond >= struct_confirm_closes:
                            exit_reason = "struct_short"
                    else:
                        if price_close < struct_stop_lvl:
                            closes_beyond += 1
                        else:
                            closes_beyond = 0
                        if closes_beyond >= struct_confirm_closes:
                            exit_reason = "struct_long"

            # --- funding-behaviour invalidation ---
            if exit_reason is None:
                # settlements since entry
                settles_since = df["is_settle"].iloc[entry_i:i+1].sum()
                if pos == -1:
                    # short: 2 settlements pinned near extreme + higher high vs entry bar
                    pinned_count = pinned_hi.iloc[entry_i:i+1].sum()
                    made_higher_high = price_high > entry_bar_high
                    if pinned_count >= 2 and made_higher_high:
                        exit_reason = "cap_persist"
                else:
                    # long: fresh multi-day low within next few bars + lower low vs entry bar
                    fresh_low_now = fresh_low.iloc[max(entry_i, i - fresh_low_lookahead + 1):i+1].any()
                    made_lower_low = price_low < entry_bar_low
                    if fresh_low_now and made_lower_low:
                        exit_reason = "fresh_low_cont"

                # --- time stop after N settlements ---
                if exit_reason is None and settles_since >= time_stop_settlements:
                    if pos == -1:
                        exit_reason = "time_short"
                    else:
                        # for long: require reclaim above entry bar high by time stop, else exit
                        if price_close <= entry_bar_high:
                            exit_reason = "time_long_no_reclaim"

            # --- exit if any reason fired ---
            if exit_reason is not None:
                # exit at close of this bar
                exit_px = price_close
                exit_time = idx[i]
                # price PnL in USD = notional * side * (exit/entry - 1)
                pnl_price_usd = notional_usd * pos * ((exit_px / entry_px) - 1.0)
                # funding CF already accumulated up to previous bar settlements; include this bar if settlement now:
                if df["is_settle"].iloc[i]:
                    fund_acc_usd += float(df[FR].iloc[i]) * notional_usd * pos
                exit_fee_usd = taker_fee * notional_usd
                pnl_total_usd = pnl_price_usd + fund_acc_usd - (pending_entry_fee + exit_fee_usd)

                trades.append({
                    "entry_time": entry_time,
                    "side": "LONG" if pos==1 else "SHORT",
                    "entry_price": entry_px,
                    "exit_time": exit_time,
                    "exit_price": exit_px,
                    "bars_held": int(i - entry_i),
                    "pnl_price_usd": float(pnl_price_usd),
                    "funding_usd": float(fund_acc_usd),
                    "fees_usd": float(-(pending_entry_fee + exit_fee_usd)),
                    "pnl_total_usd": float(pnl_total_usd),
                    "exit_reason": exit_reason
                })

                # reset position
                pos = 0
                qty = 0.0
                entry_px = np.nan
                entry_time = None
                entry_i = None
                pending_entry_fee = 0.0
                fund_acc_usd = 0.0
                closes_beyond = 0
                struct_stop_lvl = np.nan
                entry_bar_high = np.nan
                entry_bar_low  = np.nan
                next_eligible_i = i + cooldown_bars
                continue

    # close any open trade at final bar (conservative)
    if pos != 0:
        price_close = float(df[PX].iloc[-1])
        exit_px = price_close
        exit_time = df.index[-1]
        pnl_price_usd = notional_usd * pos * ((exit_px / entry_px) - 1.0)
        if df["is_settle"].iloc[-1]:
            fund_acc_usd += float(df[FR].iloc[-1]) * notional_usd * pos
        exit_fee_usd = taker_fee * notional_usd
        pnl_total_usd = pnl_price_usd + fund_acc_usd - (pending_entry_fee + exit_fee_usd)
        trades.append({
            "entry_time": entry_time,
            "side": "LONG" if pos==1 else "SHORT",
            "entry_price": entry_px,
            "exit_time": exit_time,
            "exit_price": exit_px,
            "bars_held": int(len(df)-1 - entry_i),
            "pnl_price_usd": float(pnl_price_usd),
            "funding_usd": float(fund_acc_usd),
            "fees_usd": float(-(pending_entry_fee + exit_fee_usd)),
            "pnl_total_usd": float(pnl_total_usd),
            "exit_reason": "eod_close"
        })

    log = pd.DataFrame(trades).sort_values("entry_time").reset_index(drop=True)

    # --- stats ---
    if len(log):
        pnl = log["pnl_total_usd"].values
        gp = log.loc[log["pnl_total_usd"]>0,"pnl_total_usd"].sum()
        gl = -log.loc[log["pnl_total_usd"]<0,"pnl_total_usd"].sum()
        eq = log["pnl_total_usd"].cumsum()
        max_dd = float((eq.cummax() - eq).max())
        stats = {
            "trades": int(len(log)),
            "win_rate": float((pnl>0).mean()),
            "expectancy_usd": float(np.mean(pnl)),
            "median_usd": float(np.median(pnl)),
            "profit_factor": float(gp/gl) if gl>0 else np.inf,
            "total_pnl_usd": float(np.sum(pnl)),
            "max_dd_usd": max_dd
        }
    else:
        stats = {"trades":0,"win_rate":np.nan,"expectancy_usd":np.nan,
                 "median_usd":np.nan,"profit_factor":np.nan,
                 "total_pnl_usd":0.0,"max_dd_usd":0.0}

    return log, stats


In [ ]:
# 1) parse time and set UTC datetime index
df["bar_time"] = pd.to_datetime(df["bar_time"], utc=True, errors="coerce")
df = df.set_index("bar_time").sort_index()

# 2) (only if you don't have highs/lows) create proxies from close
df["perp_high"] = df["prep_close"]
df["perp_low"]  = df["prep_close"]

# 3) run
log, stats = backtest_funding_premium_stopentry(df)
print(stats)
log.head(20)

In [ ]:
import numpy as np
import pandas as pd




def backtest_funding_multi(
    df_in: pd.DataFrame,
    # column names
    price_col="prep_close",
    high_col="perp_high",   #high of bar (we do not have both of this) -> can consider adding it
    low_col ="perp_low",    #low of bar 
    fund_col="funding_rate",
    # funding/premium
    base_per_8h = 1e-4,        # 0.01% per 8h (Binance-style)
    pct_win = 90,              # we look at funding at last 90 bars? (6 bar per day so 15 days back?)
    p_hi = 0.90, p_lo = 0.1,  # extremes as percentiles of premium (extreme percentile)
    # context windows (past-only)
    roll_high_n = 24,          # price 24-bar high for tops (4 days)
    fund_low_n  = 18,          # funding 18-bar low for bottoms (3 days)
    # stop-entry trigger
    stopentry_buffer = 0.0005, # 0.05% buffer around prior high/low # dont eneter immediately when 
    # exits (structure + safety)
    pivot_k = 6,               # swing-pivot lookback
    struct_buffer = 0.005,     # 0.5% beyond pivot
    struct_confirm_closes = 2, # 2 consecutive closes beyond stop to exit
    struct_grace_bars = 2,     # ignore structure stop first N bars
    fresh_low_lookahead = 3,   # long invalidation: fresh funding low within N bars
    time_stop_settlements = 2, # exit after N settlements if nothing else
    cat_stop_pct = 0.065,      # 6.5% catastrophe stop
    # entry throttles
    dedup_bars = 12,           # per-side de-dup (min bars between same-side entries)
    # execution & costs
    notional_usd = 10_000.0,   # fixed notional per trade
    taker_fee = 0.0004,        # 0.04% per side
    settlement_hours=(0,8,16), # 8h funding schedule (UTC)
):
    """
    Multi-position backtester: every valid signal opens a NEW independent trade.
    Returns (log: DataFrame of closed trades, stats: dict)
    """
    # ---------- prep ----------
    df = df_in.copy()

    # require datetime index
    if not pd.api.types.is_datetime64_any_dtype(df.index):
        raise ValueError("Index must be datetime (UTC). Do df.set_index('bar_time') first.")

    df = df.sort_index()

    # Ensure price/high/low columns exist (proxy high/low with price if missing)
    if high_col not in df.columns: df[high_col] = df[price_col]
    if low_col  not in df.columns: df[low_col]  = df[price_col]

    PX, HI, LO, FR = price_col, high_col, low_col, fund_col

    # settlement mask
    df["is_settle"] = df.index.hour.isin(settlement_hours) & (df.index.minute == 0)

    # funding premium & rolling percentile bands
    df["fund_premium"] = df[FR] - base_per_8h
    roll = df["fund_premium"].rolling(pct_win, min_periods=pct_win)
    df["p_hi"] = roll.quantile(p_hi)
    df["p_lo"] = roll.quantile(p_lo)

    # past-only context (NO look-ahead)
    df["roll_high_prev_24"] = df[PX].shift(1).rolling(roll_high_n, min_periods=roll_high_n).max()
    df["fund_low_prev_18"]  = df["fund_premium"].shift(1).rolling(fund_low_n, min_periods=fund_low_n).min()

    # setups (arm ideas at close of THIS bar; fill on next bar)
    setup_short = (df["fund_premium"] >= df["p_hi"]) & (df[PX] >= df["roll_high_prev_24"])
    setup_long  = (df["fund_premium"] <= df["p_lo"])  | (df["fund_premium"] <= df["fund_low_prev_18"])

    # stop-entry levels from prior bar
    prior_low  = df[LO].shift(1)
    prior_high = df[HI].shift(1)
    stop_short_lvl = prior_low  * (1 - stopentry_buffer)  # sell-stop to short
    stop_long_lvl  = prior_high * (1 + stopentry_buffer)  # buy-stop to long

    # arm now, fill on next bar if crossed
    short_fill = setup_short.shift(1) & (df[LO]  <= stop_short_lvl.shift(1))
    long_fill  = setup_long.shift(1)  & (df[HI] >= stop_long_lvl.shift(1))

    entry_px_short = stop_short_lvl.shift(1).where(short_fill)
    entry_px_long  = stop_long_lvl.shift(1).where(long_fill)

    # precompute invalidation helpers
    pinned_hi = df["is_settle"] & (df["fund_premium"] >= df["p_hi"] - 1e-12)
    fresh_low = df["fund_premium"] < df["fund_low_prev_18"] - 1e-12

    # small helpers
    def _pivot_high(i):
        j0 = max(0, i - pivot_k)
        return float(df[HI].iloc[j0:i+1].max())
    def _pivot_low(i):
        j0 = max(0, i - pivot_k)
        return float(df[LO].iloc[j0:i+1].min())

    # ---------- multi-position engine ----------
    trades = []              # closed trades
    open_trades = []         # active trades (list of dicts)
    last_entry_i_side = {"LONG": -10_000, "SHORT": -10_000}  # per-side de-dup

    idx, N = df.index, len(df)

    for i in range(1, N):
        # 1) funding accrual for every open trade at PREVIOUS settlement
        if df["is_settle"].iloc[i-1]:
            fr_prev = float(df[FR].iloc[i-1])
            for t in open_trades:
                side_mult = +1.0 if t["side"] == "LONG" else -1.0
                t["fund_usd"] += fr_prev * notional_usd * side_mult

        # 2) entries — create NEW trades even if others already exist
        if pd.notna(entry_px_short.iloc[i]) and (i - last_entry_i_side["SHORT"] >= dedup_bars):
            px_e = float(entry_px_short.iloc[i])
            open_trades.append({
                "side": "SHORT",
                "entry_px": px_e,
                "entry_i": i,
                "entry_time": idx[i],
                "entry_high": float(df[HI].iloc[i]),
                "entry_low":  float(df[LO].iloc[i]),
                "fund_usd": 0.0,
                "entry_fee": taker_fee * notional_usd,
                "closes_beyond": 0,
                "struct_stop": max(_pivot_high(i-1)*(1+struct_buffer), px_e*(1+struct_buffer/2)),
            })
            last_entry_i_side["SHORT"] = i

        if pd.notna(entry_px_long.iloc[i]) and (i - last_entry_i_side["LONG"] >= dedup_bars):
            px_e = float(entry_px_long.iloc[i])
            open_trades.append({
                "side": "LONG",
                "entry_px": px_e,
                "entry_i": i,
                "entry_time": idx[i],
                "entry_high": float(df[HI].iloc[i]),
                "entry_low":  float(df[LO].iloc[i]),
                "fund_usd": 0.0,
                "entry_fee": taker_fee * notional_usd,
                "closes_beyond": 0,
                "struct_stop": min(_pivot_low(i-1)*(1-struct_buffer), px_e*(1-struct_buffer/2)),
            })
            last_entry_i_side["LONG"] = i

        # 3) exits — evaluate each open trade
        price_close = float(df[PX].iloc[i])
        price_high  = float(df[HI].iloc[i])
        price_low   = float(df[LO].iloc[i])
        is_settle   = bool(df["is_settle"].iloc[i])

        # iterate backwards so removals are safe
        for k in range(len(open_trades)-1, -1, -1):
            t = open_trades[k]
            exit_reason = None

            # catastrophe stop (intrabar)
            if t["side"] == "SHORT" and price_high >= t["entry_px"] * (1 + cat_stop_pct):
                exit_reason = "cat_short"
            if t["side"] == "LONG" and price_low  <= t["entry_px"] * (1 - cat_stop_pct):
                exit_reason = "cat_long"

            # structure stop with grace & 2-close confirm
            if exit_reason is None and (i - t["entry_i"]) >= struct_grace_bars:
                if t["side"] == "SHORT":
                    if price_close > t["struct_stop"]:
                        t["closes_beyond"] += 1
                    else:
                        t["closes_beyond"] = 0
                    if t["closes_beyond"] >= struct_confirm_closes:
                        exit_reason = "struct_short"
                else:
                    if price_close < t["struct_stop"]:
                        t["closes_beyond"] += 1
                    else:
                        t["closes_beyond"] = 0
                    if t["closes_beyond"] >= struct_confirm_closes:
                        exit_reason = "struct_long"

            # funding-behaviour invalidation
            if exit_reason is None:
                settles_since = int(df["is_settle"].iloc[t["entry_i"]:i+1].sum())
                if t["side"] == "SHORT":
                    pinned_count = int(pinned_hi.iloc[t["entry_i"]:i+1].sum())
                    made_higher_high = price_high > t["entry_high"]
                    if pinned_count >= 2 and made_higher_high:
                        exit_reason = "cap_persist"
                else:
                    fresh_low_now = bool(fresh_low.iloc[max(t["entry_i"], i - fresh_low_lookahead + 1):i+1].any())
                    made_lower_low = price_low < t["entry_low"]
                    if fresh_low_now and made_lower_low:
                        exit_reason = "fresh_low_cont"

                # time stop after N settlements (long needs reclaim)
                if exit_reason is None and settles_since >= time_stop_settlements:
                    if t["side"] == "SHORT":
                        exit_reason = "time_short"
                    else:
                        if price_close <= t["entry_high"]:
                            exit_reason = "time_long_no_reclaim"

            # execute exit
            if exit_reason is not None:
                fund_usd = t["fund_usd"]
                if is_settle:
                    side_mult = +1.0 if t["side"] == "LONG" else -1.0
                    fund_usd += float(df[FR].iloc[i]) * notional_usd * side_mult

                side_mult = +1.0 if t["side"] == "LONG" else -1.0
                pnl_price_usd = notional_usd * side_mult * ((price_close / t["entry_px"]) - 1.0)
                exit_fee_usd  = taker_fee * notional_usd
                pnl_total_usd = pnl_price_usd + fund_usd - (t["entry_fee"] + exit_fee_usd)

                trades.append({
                    "entry_time": t["entry_time"],
                    "side": t["side"],
                    "entry_price": t["entry_px"],
                    "exit_time": idx[i],
                    "exit_price": price_close,
                    "bars_held": int(i - t["entry_i"]),
                    "pnl_price_usd": float(pnl_price_usd),
                    "funding_usd": float(fund_usd),
                    "fees_usd": float(-(t["entry_fee"] + exit_fee_usd)),
                    "pnl_total_usd": float(pnl_total_usd),
                    "exit_reason": exit_reason
                })
                del open_trades[k]

    # 4) close anything left at the end
    if len(open_trades):
        price_close = float(df[PX].iloc[-1])
        is_settle   = bool(df["is_settle"].iloc[-1])
        for t in open_trades:
            fund_usd = t["fund_usd"]
            if is_settle:
                side_mult = +1.0 if t["side"] == "LONG" else -1.0
                fund_usd += float(df[FR].iloc[-1]) * notional_usd * side_mult

            side_mult = +1.0 if t["side"] == "LONG" else -1.0
            pnl_price_usd = notional_usd * side_mult * ((price_close / t["entry_px"]) - 1.0)
            exit_fee_usd  = taker_fee * notional_usd
            pnl_total_usd = pnl_price_usd + fund_usd - (t["entry_fee"] + exit_fee_usd)

            trades.append({
                "entry_time": t["entry_time"],
                "side": t["side"],
                "entry_price": t["entry_px"],
                "exit_time": df.index[-1],
                "exit_price": price_close,
                "bars_held": int(len(df)-1 - t["entry_i"]),
                "pnl_price_usd": float(pnl_price_usd),
                "funding_usd": float(fund_usd),
                "fees_usd": float(-(t["entry_fee"] + exit_fee_usd)),
                "pnl_total_usd": float(pnl_total_usd),
                "exit_reason": "eod_close"
            })

    log = pd.DataFrame(trades).sort_values("entry_time").reset_index(drop=True)

    # ---------- stats ----------
    if len(log):
        pnl = log["pnl_total_usd"].values
        gp = log.loc[log["pnl_total_usd"]>0,"pnl_total_usd"].sum()
        gl = -log.loc[log["pnl_total_usd"]<0,"pnl_total_usd"].sum()
        eq = log["pnl_total_usd"].cumsum()
        max_dd = float((eq.cummax() - eq).max())
        stats = {
            "trades": int(len(log)),
            "win_rate": float((pnl>0).mean()),
            "expectancy_usd": float(np.mean(pnl)),
            "median_usd": float(np.median(pnl)),
            "profit_factor": float(gp/gl) if gl>0 else np.inf,
            "total_pnl_usd": float(np.sum(pnl)),
            "max_dd_usd": max_dd,
        }
    else:
        stats = {"trades":0, "win_rate":np.nan, "expectancy_usd":np.nan,
                 "median_usd":np.nan, "profit_factor":np.nan,
                 "total_pnl_usd":0.0, "max_dd_usd":0.0}

    return log, stats


In [ ]:
# 1) load your CSV once (example)
df = pd.read_csv('/Users/duncanwan/Desktop/learning/Bitcoin/4hrs/BTC_combined_2024_v2.csv')


# if your file has a bar_time column:
df["bar_time"] = pd.to_datetime(df["bar_time"], utc=True)
df = df.set_index("bar_time").sort_index()

# 2) run
log, stats = backtest_funding_multi(df, notional_usd=10_000.0)
print(stats)
log.head(10)




In [ ]:
log.groupby("side")[["pnl_price_usd","funding_usd","pnl_total_usd"]].sum()


In [ ]:
log.assign(month=log["entry_time"].dt.to_period("M")).groupby("month")["pnl_total_usd"].sum()

In [ ]:
eq = log.sort_values("exit_time")["pnl_total_usd"].cumsum()
max_dd = (eq.cummax() - eq).max()
eq.tail(), float(max_dd)


In [ ]:
log["side"].value_counts(), log["exit_reason"].value_counts().head()


In [ ]:
log["bars_held"].describe()

In [ ]:
cols = ["entry_time","exit_time","side","entry_price","exit_price",
        "bars_held","pnl_price_usd","funding_usd","fees_usd","pnl_total_usd"]
top10 = log.sort_values("pnl_total_usd", ascending=False).head(10)[cols]

worst10 = log.sort_values("pnl_total_usd", ascending=True).head(10)[cols]
print(worst10)
pd.options.display.float_format = "{:,.2f}".format
display(top10.reset_index(drop=True))
display(worst10.reset_index(drop=True))